# Cuestionario 4

Autor: Sergio Santiago Sánchez

Durante el siguiente estudio trateremos de diseñar una red que sea capaz de determinar cuando una seta es venenosa y cuando no lo es mediante una serie de caracteristicas. El dataset usado para el entrenamiento puede encontrarse [aqui](https://archive.ics.uci.edu/dataset/73/mushroom)

Usaremos las siguientes librerías:
* [Scikit-learn](https://scikit-learn.org/stable/)
* [Numpy](https://numpy.org/)
* [Pandas](https://pandas.pydata.org/)
* [Matplotlib](https://matplotlib.org/)
* [Seaborn](https://seaborn.pydata.org/)

Intalamos e importamos las librerias necesarias:

In [27]:
pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Tal y como se sugiere en la asignatura haremos uso de una versión modificada de la librería `mglearn`. Desafortunadamente, emplear esta librería nos obliga a modificar la versión de `joblib`, estableciendo esta en la 1.1.1, por lo que debido a otra imcompatibilidad debemos usar `numpy` con la versión 1.26.4, y `scikit-learn` con la versión `1.3.2`. Todas las instalaciones necesarios estan presentes en el archivo `requirements.txt`.

In [28]:
import mglearn

In [55]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scipy import stats
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

### Cargamos los datos y hacemos un estudio exploratorio preliminar

In [30]:
columns = ["poisonous", "cap-shape", "cap-surface", "cap-color", "bruises", "odor", 
           "gill-attachment", "gill-spacing", "gill-size", "gill-color", "stalk-shape", "stalk-root", "stalk-surface-above-ring", 
           "stalk-surface-below-ring", "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", "veil-color", "ring-number", "ring-type", 
           "spore-print-color", "population", "habitat"]
data = pd.read_csv("./data/agaricus-lepiota.data", names=columns)

In [31]:
display(data.head())

,poisonous,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


En este caso, la variable objetivo es representada por la primera columna `"poisonous"`, representa si el hongo es venenoso (p) o no (e). El resto son variables que permiten describir las distintas instancias.

A continuación exploraremos las distintas posibilidades para cada propiedad, para de este modo llevar a cabo una estrategia u otra. Ya que por ejemplo, si hablamos de variables categóricas que implican un cierto orden, podriamos sustituir los valores por enteros, por el contraria, si las variables no reflejan ningún tipo de orden podríamos optar por usar [`one hot`](https://interactivechaos.com/es/manual/tutorial-de-machine-learning/one-hot-encoding).

La técnica de one hot, consiste en añadir una columna al dataset por cada posibilidad de la variable a reemplazar. En cada instancia de los datos asignaremos un 0 en aquellas columnas extra que no coincidan con su valor original y un 1 en aquella que si represente la asignación original.

Usar una técnica u otra ayuda al modelo a no malinterpretar relaciones, ya que si sustituimos siempre por valores numéricos podría llegar a pensar que una cualidad esta por encima de otra. Por otro lado, si siempre aplicamos one hot, privamos al modelo de aprender esa posible relación en los casos en los que sí tenga sentido.

In [32]:
for col in data.columns:
    print( f"Column {col:<25}: {data[col].unique()}")

Column poisonous                : ['p' 'e']
Column cap-shape                : ['x' 'b' 's' 'f' 'k' 'c']
Column cap-surface              : ['s' 'y' 'f' 'g']
Column cap-color                : ['n' 'y' 'w' 'g' 'e' 'p' 'b' 'u' 'c' 'r']
Column bruises                  : ['t' 'f']
Column odor                     : ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
Column gill-attachment          : ['f' 'a']
Column gill-spacing             : ['c' 'w']
Column gill-size                : ['n' 'b']
Column gill-color               : ['k' 'n' 'g' 'p' 'w' 'h' 'u' 'e' 'b' 'r' 'y' 'o']
Column stalk-shape              : ['e' 't']
Column stalk-root               : ['e' 'c' 'b' 'r' '?']
Column stalk-surface-above-ring : ['s' 'f' 'k' 'y']
Column stalk-surface-below-ring : ['s' 'f' 'y' 'k']
Column stalk-color-above-ring   : ['w' 'g' 'p' 'n' 'b' 'e' 'o' 'c' 'y']
Column stalk-color-below-ring   : ['w' 'p' 'g' 'b' 'n' 'e' 'y' 'o' 'c']
Column veil-type                : ['p']
Column veil-color               : ['w' 'n' 'o' '

Tal y como podemos observar, existen algunos valores nulos, los cuales han sido representados mediante `"?"`. En primer lugar observaremos si hay algun valor nulo más y luego probaremos a cambiarlos por `"Unknow"`.

In [33]:
data.isnull().sum()

poisonous                   0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [34]:
data = data.replace("?", "UNKNOW")

Vamos a distribuir las columnas de clasificaciones segun sean binarias o múltiples.

In [11]:
binary_classification = []
multiple_clasification = []

for col in data.columns:
    
    if(len(data[col].unique()) > 2):
        multiple_clasification.append(col)
    else:
        binary_classification.append(col)

print(f"Binary classification: {binary_classification}")
print(f"Multiple classification: {multiple_clasification}")

Binary classification: ['poisonous', 'bruises', 'gill-attachment', 'gill-spacing', 'gill-size', 'stalk-shape', 'veil-type']
Multiple classification: ['cap-shape', 'cap-surface', 'cap-color', 'odor', 'gill-color', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']


A continuación, vamos a mapear las clasificaciones binarias en 0 y 1. De este modo permitimos que la red sea capaz de aprender de dichas variables.

In [12]:
data_binary_mapped = data.copy()
for col in binary_classification:
    mapping = dict(zip(data_binary_mapped[col].unique(), range(2)))
    data_binary_mapped[col] = data_binary_mapped[col].map(mapping)
    print(f"Column {col} mapped -> {data_binary_mapped[col].unique()}")

Column poisonous mapped -> [0 1]
Column bruises mapped -> [0 1]
Column gill-attachment mapped -> [0 1]
Column gill-spacing mapped -> [0 1]
Column gill-size mapped -> [0 1]
Column stalk-shape mapped -> [0 1]
Column veil-type mapped -> [0]


Ahora debemos mapear o aplicar one hot al resto de columnas, para decidir la operación a aplicar debemos fijarnos previamente en la naturaleza de los datos:

| **Inglés**                     | **Español**                      | **Valores posibles**                                                                                     |
|--------------------------------|-----------------------------------|----------------------------------------------------------------------------------------------------------|
| cap-shape                      | forma del sombrero               | bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s                                                |
| cap-surface                    | superficie del sombrero          | fibrous=f, grooves=g, scaly=y, smooth=s                                                                 |
| cap-color                      | color del sombrero               | brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y                |
| odor                           | olor                             | almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s                     |
| gill-color                     | color de las laminillas          | black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u, red=e, white=w, yellow=y |
| stalk-root                     | raíz del tallo                   | bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=UNKNOW                                   |
| stalk-surface-above-ring       | superficie del tallo sobre el anillo | fibrous=f, scaly=y, silky=k, smooth=s                                                                  |
| stalk-surface-below-ring       | superficie del tallo debajo del anillo | fibrous=f, scaly=y, silky=k, smooth=s                                                              |
| stalk-color-above-ring         | color del tallo sobre el anillo  | brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y                        |
| stalk-color-below-ring         | color del tallo debajo del anillo | brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y                       |
| veil-color                     | color del velo                   | brown=n, orange=o, white=w, yellow=y                                                                    |
| ring-number                    | número de anillos                | none=n, one=o, two=t                                                                                    |
| ring-type                      | tipo de anillo                   | cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z                   |
| spore-print-color              | color de la impresión de esporas | black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y                  |
| population                     | población                        | abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y                                |
| habitat                        | hábitat                          | grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d                                     |



Tras observar los atributos restantes y sus posibles valores podemos afirmar que `"ring-number"` (número de anillos) y `"population"` (población), son las únicas propiedades que contienen un orden ímplicito. Por ello, aplicaremos un reemplazo por valores numéricos a esas variables, mientras que al resto aplicaremos one hot.

In [13]:
number_clasification = ["ring-number", "population"]
ring_number_map = {
    'n': 0,
    'o': 1,
    't': 2
    }
population_map = {
    'y': 0,
    'v': 1,
    's': 2,
    'n': 3,
    'c': 4,
    'a': 5
    }
one_hot_classification = [i for i in multiple_clasification if i not in number_clasification]

In [14]:
data_mapped = data_binary_mapped.copy()

data_mapped["ring-number"] = data_mapped["ring-number"].map(ring_number_map)
data_mapped["population"] = data_mapped["population"].map(population_map)

In [15]:
data_mapped = pd.get_dummies(data_mapped, columns=one_hot_classification, dtype=int)

Ahora, tras haber hecho aplicado estas operaciones sobre los datos, realizaremos otro visionado de los mismos, para comprobar que todo se ha aplicado de forma correcta.

In [16]:
display(data_mapped)
for col in data_mapped.columns:
    print( f"Column {col:<25}: {data_mapped[col].unique()}")

,poisonous,bruises,gill-attachment,gill-spacing,gill-size,stalk-shape,veil-type,ring-number,population,cap-shape_b,...,spore-print-color_u,spore-print-color_w,spore-print-color_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,1,0,0,1,3,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,1,0,0,1,3,1,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,1,0
4,1,1,0,1,1,1,0,1,5,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,1,1,1,0,1,0,0,1,4,0,...,0,0,0,0,0,1,0,0,0,0
8120,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0
8121,1,1,1,0,1,0,0,1,4,0,...,0,0,0,0,0,1,0,0,0,0
8122,0,1,0,0,0,1,0,1,1,0,...,0,1,0,0,0,1,0,0,0,0


Column poisonous                : [0 1]
Column bruises                  : [0 1]
Column gill-attachment          : [0 1]
Column gill-spacing             : [0 1]
Column gill-size                : [0 1]
Column stalk-shape              : [0 1]
Column veil-type                : [0]
Column ring-number              : [1 2 0]
Column population               : [2 3 5 1 0 4]
Column cap-shape_b              : [0 1]
Column cap-shape_c              : [0 1]
Column cap-shape_f              : [0 1]
Column cap-shape_k              : [0 1]
Column cap-shape_s              : [0 1]
Column cap-shape_x              : [1 0]
Column cap-surface_f            : [0 1]
Column cap-surface_g            : [0 1]
Column cap-surface_s            : [1 0]
Column cap-surface_y            : [0 1]
Column cap-color_b              : [0 1]
Column cap-color_c              : [0 1]
Column cap-color_e              : [0 1]
Column cap-color_g              : [0 1]
Column cap-color_n              : [1 0]
Column cap-color_p              

Aunque la mayoría de los datos presentan un rango entre 0 y 1, hay algunas variables que no siguen un rango, es por ello que vamos a aplciar una estandarización de los datos. De este modo estamos ayudando a la red neuronal a obtener los valores optimos para los pesos, ya que técnicas como el descenso del gradiente se ven beneficidas de este prepocesamiento. En concreto, mediante [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) permite mantener una media de 0 y una desviación estandar de 1, con lo que conseguimos tratar a todos los valores por igual.

In [17]:
scaler = StandardScaler()
columns_to_scale = ['ring-number', 'population']
data_mapped[columns_to_scale] = scaler.fit_transform(data_mapped[columns_to_scale])
for col in data_mapped.columns:
    print( f"Column {col:<25}: {data_mapped[col].unique()}")

Column poisonous                : [0 1]
Column bruises                  : [0 1]
Column gill-attachment          : [0 1]
Column gill-spacing             : [0 1]
Column gill-size                : [0 1]
Column stalk-shape              : [0 1]
Column veil-type                : [0]
Column ring-number              : [-0.25613174  3.43325525 -3.94551873]
Column population               : [ 0.5143892   1.31310821  2.91054623 -0.28432981 -1.08304882  2.11182722]
Column cap-shape_b              : [0 1]
Column cap-shape_c              : [0 1]
Column cap-shape_f              : [0 1]
Column cap-shape_k              : [0 1]
Column cap-shape_s              : [0 1]
Column cap-shape_x              : [1 0]
Column cap-surface_f            : [0 1]
Column cap-surface_g            : [0 1]
Column cap-surface_s            : [1 0]
Column cap-surface_y            : [0 1]
Column cap-color_b              : [0 1]
Column cap-color_c              : [0 1]
Column cap-color_e              : [0 1]
Column cap-color_g    

## Entrenando la red

El siguiente paso es entrenar una red neuronal para poder realizar predicciones. Para ello debemos sepaar el conjunto de datos en entrenamiento y pruebas. En primer lugar debemos separar los datos según son variables del posible modelo o características a predecir.

In [18]:
x_data = data_mapped.iloc[:, 1:]
y_data = data_mapped.iloc[:, 0:1]

In [19]:
display(x_data.head())
display(y_data.head())

,bruises,gill-attachment,gill-spacing,gill-size,stalk-shape,veil-type,ring-number,population,cap-shape_b,cap-shape_c,...,spore-print-color_u,spore-print-color_w,spore-print-color_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,0,-0.256132,0.514389,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,-0.256132,1.313108,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,-0.256132,1.313108,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,-0.256132,0.514389,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,1,1,1,0,-0.256132,2.910546,0,0,...,0,0,0,0,1,0,0,0,0,0


,poisonous
0,0
1,1
2,1
3,0
4,1


Mediante el método [train_test_split](https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.train_test_split.html) realizamos una división del dataset en dos conjuntos, entrenamiento y pruebas.

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=557)

Definimos una función que nos permita entrenar modelos con distintos hiperparámetros de form sencilla. Para entrenar la red usaremos [`MLPClassifier`](https://scikit-learn.org/1.5/modules/generated/sklearn.neural_network.MLPClassifier.html) de la librería de Scikit-learn.

In [58]:
def develop_net(f_act, layers, x_train_data: pd.DataFrame = x_train, y_train_data: pd.DataFrame = y_train, x_test_data: pd.DataFrame = x_test, y_test_data: pd.DataFrame = y_test):    
    x_data_total = pd.concat([x_train_data, x_test_data], axis=0)
    y_data_total = pd.concat([y_train_data, y_test_data], axis=0)
    
    y_train_data = y_train_data.values.ravel()
    y_test_data = y_test_data.values.ravel()
    y_data_total = y_data_total.values.ravel()
    
    mlp_model = MLPClassifier(solver='lbfgs',
                               random_state=0,
                               hidden_layer_sizes=layers,
                               activation=f_act,
                               max_iter=1000)
    mlp_model.fit(x_train_data, y_train_data)
    
    y_test_pred = mlp_model.predict(x_test_data)
    
    print(f"train data shape:  x -> {x_train_data.shape} y -> {y_train_data.shape}")
    print(f"test data shape:  x -> {x_test_data.shape} y -> {y_test_data.shape}")
    print(f"total data shape:  x -> {x_data_total.shape} y -> {y_data_total.shape}")
    print("Rendimiento en el conjunto de entrenamiento: ",mlp_model.score(x_train_data,y_train_data))
    print("Rendimiento en el conjunto de prueba: ",mlp_model.score(x_test_data,y_test_data))
    print("Rendimiento en el conjunto total: ",mlp_model.score(x_data_total,y_data_total))
    
    print("\nReporte de clasificación en conjunto de prueba:\n")
    print(classification_report(y_test_data, y_test_pred))
    
    return mlp_model

In [60]:
develop_net('relu',(1,2,))

train data shape:  x -> (5686, 105) y -> (5686,)
test data shape:  x -> (2438, 105) y -> (2438,)
total data shape:  x -> (8124, 105) y -> (8124,)
Rendimiento en el conjunto de entrenamiento:  0.992085824832923
Rendimiento en el conjunto de prueba:  0.9934372436423298
Rendimiento en el conjunto total:  0.9924913835548991

Reporte de clasificación en conjunto de prueba:

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1164
           1       1.00      0.99      0.99      1274

    accuracy                           0.99      2438
   macro avg       0.99      0.99      0.99      2438
weighted avg       0.99      0.99      0.99      2438



MLPClassifier(hidden_layer_sizes=(1, 2), max_iter=1000, random_state=0,
              solver='lbfgs')